Week 1 Homework Related

In [4]:
import pandas as pd

In [6]:
## question 1
!curl localhost:9200

{
  "name" : "492ef111da64",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "jO_N5Kv6RVqn6KQEYxhECg",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [9]:
## Question 2
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [14]:
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

/home/antihaddock/anaconda3/envs/llmzoomcamp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [16]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:07<00:00, 127.64it/s]


In [ ]:
## Question 3

In [33]:
# Define the search function
def elastic_search(query, size=5, course=None):
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text", "section"],
                        "type": "best_fields"
                    }
                }#,
                # "filter": {
                #     "term": {
                #         "course": "data-engineering-zoomcamp"
                # }
                #}
            }
        }
    }

    # Execute the search query
    response = es_client.search(index=index_name, body=search_query)

    # Extract the score of the top hit - new code
    top_score = None
    if response['hits']['hits']:
        top_score = response['hits']['hits'][0]['_score']
    
    # Extract the result documents
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return top_score, result_docs

In [38]:
def elastic_search(query, size=5, course=None):
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text", "section"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    if course:
        search_query['query']['bool']['filter'] = {
            "term": {
                "course": course
            }
        }

    # Execute the search query
    response = es_client.search(index=index_name, body=search_query)

    # Extract the score of the top hit
    top_score = None
    if response['hits']['hits']:
        top_score = response['hits']['hits'][0]['_score']
    
    # Extract the result documents
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return top_score, result_docs

In [41]:
query = 'How do I execute a command in a running docker container?'

top_score, result_docs = elastic_search(query)
print("Top score:", top_score)
print("Result documents:", result_docs)

Top score: 84.050095
Result documents: [{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}, {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\nPassword for root:

In [42]:
result_docs

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\nPassword for root:\nServer: PostgreSQL 16.1 (Debia

In [52]:
## Question 4
# Limit to machine-learning-zoomcamp.
# Return only 3 results

query = 'How do I execute a command in a running docker container?'

top_score, result_docs = elastic_search(query, size=3, course='machine-learning-zoomcamp')

print(result_docs[2]['question'])

How do I copy files from a different folder into docker container’s working directory?


In [59]:
#Question 5

#Update provided prompt function with new context and prompt template 

def build_prompt(query, search_results):
    # New context template
    context_template = """
Q: {question}
A: {text}
""".strip()

    # New prompt template
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    # Construct the context using the new context template
    context = ""
    for doc in search_results:
        context += context_template.format(question=doc['question'], text=doc['text']) + "\n\n"
    
    # Construct the prompt using the new prompt template
    prompt = prompt_template.format(question=query, context=context.strip())

    
    return prompt

In [62]:
_, search_results = elastic_search(query, size=3, course='machine-learning-zoomcamp')
prompt = build_prompt(query, search_results)
len(prompt)

1462

In [63]:
## Question 6
!pip install tiktoken


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 196.1 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 1.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.1/775.1 kB 2.0 MB/s eta 0:00:0000:0100:01


In [67]:
import tiktoken

# Get the encoding for the GPT-4o model
encoding = tiktoken.encoding_for_model("gpt-4o")
    
# Encode the prompt and count the number of tokens
tokenized_prompt = encoding.encode(prompt)
num_tokens = len(tokenized_prompt)
print(num_tokens)

322
